# Ivestment Porfolio Management

## Load investments and print short report

In [ ]:
import sys
import os
import logging
import pandas
import numpy

logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

# Give priority to local packages (not needed in case Robson was installed by pip)
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(''), '..')))

import investor

In [ ]:
# Use data from cache
me=investor.Investor("../investor_ui_config.yaml")

# # Pass a refreshMap to force refresh data from the Internet
# refresh=dict(
#     portfolio=True,
#     benchmarks=False,
#     currency_converters=False
# )

# me=investor.Investor(
#     "../investor_ui_config.yaml",
#     refreshMap=refresh #dict(zip(investor.Investor.domains,len(investor.Investor.domains)*[True]))
# )

In [ ]:
me.config

## Get list of investment instruments and currencies

In [ ]:
me.portfolio.funds()

## Get compound fund from a few instruments

In [ ]:
me.exchange.currency='USD'

In [ ]:
# fund=me.portfolio.getFund(currencyExchange=me.exchange)
fund=me.portfolio.getFund(['TraderBot KuCoin'],currencyExchange=me.exchange)
fund

## Get current Balance of all instruments

In [ ]:
currentBalance=fund.balance.groupby(level=0).last()
currentBalance.columns=currentBalance.columns.droplevel()
currentBalance=currentBalance.sort_values(me.exchange.target,ascending=False)
currentBalance

### Current Balance of only the top 70% of Portfolio

In [ ]:
qcut=0.7

(
    currentBalance
    .sort_values(me.exchange.target,ascending=False)
    .assign(
        cum=currentBalance[me.exchange.target].cumsum()
    )
    .query("cum<{}".format(currentBalance[me.exchange.target].sum()*qcut))
    [[me.exchange.target]]
)

## Compute periodic report

Pass a list to `kpi` parameter to show only specific KPIs. Avaialble are:

**Source of all information**
* `investor.KPI.BALANCE`\
plain balance
* `investor.KPI.MOVEMENTS`\
money added and removed from fund on period

**Cumulative movements**
* `investor.KPI.SAVINGS`\
cumulative money added and removed from fund

**Rate of accumulated gains**
* `investor.KPI.BALANCE_OVER_SAVINGS`\
balance ➗ savings

**Pure gain, on the period or accumulated**
* `investor.KPI.GAINS`\
cumulative gains
* `investor.KPI.PERIOD_GAIN`\
gain on each period

**Normalization**
* `investor.KPI.SHARES`\
amount of shares
* `investor.KPI.SHARE_VALUE`\
value of a share

**Performance**
* `investor.KPI.RATE_RETURN`\
percentage change of share value

**KPIs related to external sources**
* `investor.KPI.BENCHMARK`\
raw value of the benchmark
* `investor.KPI.BENCHMARK_RATE_RETURN`\
variation of benchmark in relation to the last period
* `investor.KPI.BENCHMARK_EXCESS_RETURN`\
relation between RATE_RETURN and BENCHMARK_RATE_RETURN, on each period

### Month and Year

In [ ]:
fund.report(benchmark=me.benchmarks[8]['obj'])

### Quarter & Year

In [ ]:
fund.report('Q',benchmark=me.benchmarks[9]['obj'])

### Daily

In [ ]:
fund.report('D',benchmark=me.benchmarks[2]['obj'], output='flat')

#### Plain daily gains

In [ ]:
me.benchmarks[8]

In [ ]:
fund.report('D',benchmark=me.benchmarks[8]['obj'],kpi=[investor.KPI.RATE_RETURN,investor.KPI.BALANCE_OVER_SAVINGS,investor.KPI.BENCHMARK,investor.KPI.BENCHMARK_RATE_RETURN])

In [ ]:
fund.report('D',benchmark=me.benchmarks[2]['obj'],kpi=[investor.KPI.PERIOD_GAIN,investor.KPI.GAINS,investor.KPI.RATE_RETURN,investor.KPI.BALANCE_OVER_SAVINGS])

In [ ]:
fund.periodicReport('M',benchmark=me.benchmarks[8]['obj'])

### Week & 4 Weeks
#### Performance report

In [ ]:
# fund=me.portfolio[0]['obj'].getFund(['ShiguBot Binance','ShiguBot MB'],currencyExchange=me.exchange)
fund=me.portfolio.getFund(['TraderBot KuCoin'],currencyExchange=me.exchange)

fund.report(
    period='W',
    benchmark=me.benchmarks[8]['obj'],
    kpi=[
        investor.KPI.RATE_RETURN,
        investor.KPI.BENCHMARK_RATE_RETURN,
        investor.KPI.BENCHMARK_EXCESS_RETURN,
        investor.KPI.PERIOD_GAIN
    ],
)

#### Wealth Evolution

In [ ]:
fund.report(
    period='W',
    benchmark=me.benchmarks[9]['obj'],
    kpi=[
        investor.KPI.BALANCE,
        investor.KPI.BALANCE_OVER_SAVINGS,
        investor.KPI.GAINS,
        investor.KPI.SAVINGS,
        investor.KPI.MOVEMENTS
    ],
)

#### Plain weekly gains

In [ ]:
fund.report('W',benchmark=me.benchmarks[9]['obj'],kpi=[investor.KPI.PERIOD_GAIN])

#### Plain Periodic Report

In [ ]:
pandas.concat([l], axis=1, keys=[fund.periodicReport('4W').index[0]])

----